<a href="https://colab.research.google.com/github/GuilhermeBorba2/non-linear-regression/blob/main/Regress%C3%A3o_n%C3%A3o_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.io.sql import DatabaseError
import statsmodels.api as sm
import statsmodels.formula.api as smf
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import export_graphviz
import pydot




In [2]:
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [3]:
df = pd.read_excel('/content/gdrive/MyDrive/pasta_estudo /temps.xlsx')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual
0,2016,1,1,Fri,45,45,45.6,45
1,2016,1,2,Sat,44,45,45.7,44
2,2016,1,3,Sun,45,44,45.8,41
3,2016,1,4,Mon,44,41,45.9,40
4,2016,1,5,Tues,41,40,46.0,44


In [4]:
df.describe()

,year,month,day,temp_2,temp_1,average,actual
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     348 non-null    int64  
 1   month    348 non-null    int64  
 2   day      348 non-null    int64  
 3   week     348 non-null    object 
 4   temp_2   348 non-null    int64  
 5   temp_1   348 non-null    int64  
 6   average  348 non-null    float64
 7   actual   348 non-null    int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 21.9+ KB


In [7]:
df = pd.get_dummies(df)
df.head()

,year,month,day,temp_2,temp_1,average,actual,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,0,0,0,0,0,1,0


In [8]:
#transformando em ARRAY
labels = np.array(df["actual"])
df = df.drop("actual", axis=1)
df_list = list(df.columns)
df =np.array(df)

In [11]:
train_df, test_df, train_labels, test_labels = train_test_split(df, labels, test_size=0.25, random_state=42)

In [13]:
baseline_preds = test_df[:, df_list.index("average")]
baseline_error = abs(baseline_preds - test_labels)
print("Baseline error average: ", round(np.mean(baseline_error),2))

Baseline error average:  5.06


In [18]:
#RANDOMFORESTREGRESSOR
rf= RandomForestRegressor(n_estimators=1000, random_state=42)
rf.fit(train_df, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [20]:
prediction_rf = rf.predict(test_df)
error_rf =abs( prediction_rf - test_labels) 

r_sq = rf.score(df, labels)
print("R^2: ", r_sq)
print("MAE: ", metrics.mean_absolute_error(test_labels, prediction_rf))
print("MSE: ", metrics.mean_squared_error(test_labels, prediction_rf))



R^2:  0.932094797587982
MAE:  3.932057471264368
MSE:  26.68358100000001


In [22]:
#ADABOOST
ada = AdaBoostRegressor(n_estimators=100)
ada.fit(train_df, train_labels)

ada_pred = ada.predict(test_df)
error_ada = abs(ada_pred - test_labels)

r_sq = ada.score(df, labels)
print("R^2: ", r_sq)
print("MAE: ", metrics.mean_absolute_error(test_labels, ada_pred))
print("MSE: ", metrics.mean_squared_error(test_labels, ada_pred))



R^2:  0.8810507786538954
MAE:  3.617888126813807
MSE:  23.093760611766218


In [23]:
#GradiantBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=100)
gbr.fit(train_df, train_labels)

gbr_pred = gbr.predict(test_df)
error_gbr = abs(gbr_pred - test_labels)

r_sq = gbr.score(df, labels)
print("R^2: ", r_sq)
print("MAE: ", metrics.mean_absolute_error(test_labels, gbr_pred))
print("MSE: ", metrics.mean_squared_error(test_labels, gbr_pred))



R^2:  0.9175861622909427
MAE:  4.07948570493565
MSE:  28.51243129895993


In [28]:
rf = RandomForestRegressor(max_depth=3)
rf.fit(train_df, train_labels)

tree = rf.estimators_[5]

tree

export_graphviz(tree, out_file="tree.dot", feature_names = df_list, rounded=True, precision=1)
(graph, ) = pydot.graph_from_dot_file("tree.dot")

graph.write_png("tree.png")


In [30]:
importances = list(rf.feature_importances_)
feature_importance = [(df, round(importance,2)) for df, importance in zip(df_list, importances)]

feature_importance = sorted(feature_importance, key = lambda x:[1], reverse =True)

[print("Feature: {:20} Importance {}".format(*pair)) for pair in feature_importance];


Feature: year                 Importance 0.0
Feature: month                Importance 0.0
Feature: day                  Importance 0.0
Feature: temp_2               Importance 0.01
Feature: temp_1               Importance 0.78
Feature: average              Importance 0.2
Feature: week_Fri             Importance 0.0
Feature: week_Mon             Importance 0.0
Feature: week_Sat             Importance 0.0
Feature: week_Sun             Importance 0.0
Feature: week_Thurs           Importance 0.0
Feature: week_Tues            Importance 0.0
Feature: week_Wed             Importance 0.0


In [31]:
importances = list(ada.feature_importances_)
feature_importance = [(df, round(importance,2)) for df, importance in zip(df_list, importances)]

feature_importance = sorted(feature_importance, key = lambda x:[1], reverse =True)

[print("Feature: {:20} Importance {}".format(*pair)) for pair in feature_importance];

Feature: year                 Importance 0.0
Feature: month                Importance 0.08
Feature: day                  Importance 0.03
Feature: temp_2               Importance 0.08
Feature: temp_1               Importance 0.5
Feature: average              Importance 0.25
Feature: week_Fri             Importance 0.01
Feature: week_Mon             Importance 0.04
Feature: week_Sat             Importance 0.0
Feature: week_Sun             Importance 0.01
Feature: week_Thurs           Importance 0.0
Feature: week_Tues            Importance 0.0
Feature: week_Wed             Importance 0.0


In [35]:
importances = list(gbr.feature_importances_)
feature_importance = [(df, round(importance,2)) for df, importance in zip(df_list, importances)]

feature_importance = sorted(feature_importance, key = lambda x:[1], reverse =True)

[print("Feature: {:20} Importance {}".format(*pair)) for pair in feature_importance];

Feature: year                 Importance 0.0
Feature: month                Importance 0.01
Feature: day                  Importance 0.02
Feature: temp_2               Importance 0.01
Feature: temp_1               Importance 0.63
Feature: average              Importance 0.3
Feature: week_Fri             Importance 0.01
Feature: week_Mon             Importance 0.0
Feature: week_Sat             Importance 0.0
Feature: week_Sun             Importance 0.0
Feature: week_Thurs           Importance 0.0
Feature: week_Tues            Importance 0.0
Feature: week_Wed             Importance 0.0
